# PaddleSeg动态图API使用教程

本教程旨在帮助大家快速掌握PaddleSeg的API调用，轻松进行语义分割模型的训练、验证和预测。

如果对配置化调用方式更感兴趣，可参考[快速上手PaddleSeg文档](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.7/docs/whole_process_cn.md)。

Note：
* 请在AI studio上fork本项目的最新版本，然后运行使用。
* 若想更详细地了解PaddleSeg API，请阅读[API文档](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.7/docs/apis/README_CN.md)。

In [3]:
%cd /root/codes/ocr/paddle-ocr-learn/paddle_learn/lesson09/PaddleSeg

/root/codes/ocr/paddle-ocr-learn/paddle_learn/lesson09/PaddleSeg


# 一、环境准备

**PaddlePaddle环境准备**

如果在AI Studio上运行此项目，请选择使用GPU版本的环境，默认已经安装了PaddlePaddle。
    
如果在本地运行此教程，需要自行安装PaddlePaddle。
由于图像分割模型计算开销大，建议在GPU版本的PaddlePaddle下使用PaddleSeg，安装教程请见[PaddlePaddle官网](https://www.paddlepaddle.org.cn/)。推荐：
* PaddlePaddle （最新版本）
* Python = 3.7

**PaddleSeg环境准备**

安装过程请参考[PaddleSeg安装文档](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.7/docs/install_cn.md)，下面进行安装：

# 二、模型训练、验证和预测

本教程使用PP_LiteSeg模型在视盘分割数据集上基于API进行开发。


## 2.1 模型训练

使用API进行训练的过程可分为五步，分别是构建模型、构建训练集与验证集、构建优化器、构建损失函数和启动训练。

### 构建模型

PaddleSeg在paddleseg.models子模块中提供了常用的分割模型，在paddleseg.models.backbones子模块中提供了分割模型常用的骨干网络，大家可以直接使用。

In [4]:
from paddleseg.models import PPLiteSeg
from paddleseg.models.backbones import STDC1
model = PPLiteSeg(
    num_classes=2, # 背景+前景类别数
    backbone=STDC1(pretrained='https://bj.bcebos.com/paddleseg/dygraph/PP_STDCNet1.tar.gz')
)

2024-02-13 20:09:13 [INFO]	Loading pretrained model from https://bj.bcebos.com/paddleseg/dygraph/PP_STDCNet1.tar.gz


W0213 20:09:13.637141 978575 gpu_resources.cc:119] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.2, Runtime API Version: 12.0
W0213 20:09:13.638809 978575 gpu_resources.cc:164] device: 0, cuDNN Version: 8.9.


Connecting to https://bj.bcebos.com/paddleseg/dygraph/PP_STDCNet1.tar.gz
[==================================================] 100.00%
Uncompress PP_STDCNet1.tar.gz
[==================================================] 100.00%
2024-02-13 20:09:17 [INFO]	There are 145/145 variables loaded into STDCNet.


### 构建训练集与验证集

PaddleSeg在paddleseg.transforms子模块中提供了常用的数据预处理操作，需要注意的是PaddleSeg默认会添加读取图像操作、HWC转CHW的操作，所以这两个操作不用添加到transforms列表中。

在paddleseg.datasets子模块中提供了分割常用的数据集构建API，也支持符合格式的[自定义数据集](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.7/docs/data/marker/marker_cn.md)，本教程使用的是子模块提供的数据集API。

In [5]:
# 训练集预处理操作
import paddleseg.transforms as T
train_transforms = [
    T.Resize(target_size=(512, 512)),
    T.RandomHorizontalFlip(),
    T.Normalize()
]

# 构建训练集
from paddleseg.datasets import OpticDiscSeg
train_dataset = OpticDiscSeg(
    dataset_root='data/optic_disc_seg',
    transforms=train_transforms,
    mode='train'
)

Connecting to https://paddleseg.bj.bcebos.com/dataset/optic_disc_seg.zip
[==================================================] 100.00%
Uncompress optic_disc_seg.zip
[==================================================] 100.00%


In [6]:
# 验证集预处理操作
import paddleseg.transforms as T
val_transforms = [
    T.Resize(target_size=(512, 512)),
    T.Normalize()
]

# 构建验证集
from paddleseg.datasets import OpticDiscSeg
val_dataset = OpticDiscSeg(
    dataset_root='data/optic_disc_seg',
    transforms=val_transforms,
    mode='val'
)

### 构建优化器

Paddle框架提供了丰富的[学习率策略API和优化器API](https://www.paddlepaddle.org.cn/documentation/docs/zh/api/paddle/optimizer/Overview_cn.html)，本教程使用PolynomialDecay学习率策略，用法如下：

In [7]:
import paddle
base_lr = 0.01
lr = paddle.optimizer.lr.PolynomialDecay(base_lr, power=0.9, decay_steps=1000, end_lr=0)
optimizer = paddle.optimizer.Momentum(lr, parameters=model.parameters(), momentum=0.9, weight_decay=4.0e-5)

### 构建损失函数

PaddleSeg在paddleseg.models.losses子模块中提供了分割模型常用的损失函数，为了适应多路损失，损失函数应构建成包含'types'和'coef'的dict，如下所示。 

其中losses['type']表示损失函数类型， losses['coef']为对应的系数。需注意len(losses['types'])应等于len(losses['coef'])。

In [8]:
from paddleseg.models.losses import OhemCrossEntropyLoss
losses = {}
losses['types'] = [OhemCrossEntropyLoss(min_kept=200000)] * 3
losses['coef'] = [1] * 3

### 启动训练

paddleseg.core提供了模型训练的API，指定参数后即可进行训练。

In [9]:
from paddleseg.core import train
train(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=optimizer,
    save_dir='output',
    iters=1000,
    batch_size=4,
    save_interval=200,
    log_iters=10,
    num_workers=0,
    losses=losses,
    use_vdl=True)

/root/miniconda3/envs/paddle_learn/lib/python3.8/site-packages/paddle/nn/layer/norm.py:824: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


2024-02-13 20:12:14 [INFO]	[TRAIN] epoch: 1, iter: 10/1000, loss: 2.0496, lr: 0.009919, batch_cost: 0.3323, reader_cost: 0.01253, ips: 12.0356 samples/sec | ETA 00:05:29
2024-02-13 20:12:14 [INFO]	[TRAIN] epoch: 1, iter: 20/1000, loss: 0.6874, lr: 0.009829, batch_cost: 0.0625, reader_cost: 0.00019, ips: 63.9732 samples/sec | ETA 00:01:01
2024-02-13 20:12:15 [INFO]	[TRAIN] epoch: 1, iter: 30/1000, loss: 0.4588, lr: 0.009739, batch_cost: 0.0641, reader_cost: 0.00022, ips: 62.4071 samples/sec | ETA 00:01:02
2024-02-13 20:12:16 [INFO]	[TRAIN] epoch: 1, iter: 40/1000, loss: 0.3216, lr: 0.009648, batch_cost: 0.0630, reader_cost: 0.00019, ips: 63.4861 samples/sec | ETA 00:01:00
2024-02-13 20:12:16 [INFO]	[TRAIN] epoch: 1, iter: 50/1000, loss: 0.2227, lr: 0.009558, batch_cost: 0.0651, reader_cost: 0.00018, ips: 61.4117 samples/sec | ETA 00:01:01
2024-02-13 20:12:17 [INFO]	[TRAIN] epoch: 1, iter: 60/1000, loss: 0.2306, lr: 0.009467, batch_cost: 0.0652, reader_cost: 0.00019, ips: 61.3717 samples

76/76 [==============================] - 1s 20ms/step - batch_cost: 0.0194 - reader cost: 4.3684e-04


2024-02-13 20:12:28 [INFO]	[EVAL] #Images: 76 mIoU: 0.8875 Acc: 0.9956 Kappa: 0.8738 Dice: 0.9369
2024-02-13 20:12:28 [INFO]	[EVAL] Class IoU: 
[0.9956 0.7794]
2024-02-13 20:12:28 [INFO]	[EVAL] Class Precision: 
[0.997  0.9176]
2024-02-13 20:12:28 [INFO]	[EVAL] Class Recall: 
[0.9986 0.8381]
2024-02-13 20:12:28 [INFO]	[EVAL] The model with the best validation mIoU (0.8875) was saved at iter 200.
2024-02-13 20:12:29 [INFO]	[TRAIN] epoch: 4, iter: 210/1000, loss: 0.0937, lr: 0.008098, batch_cost: 0.0709, reader_cost: 0.00025, ips: 56.4501 samples/sec | ETA 00:00:55
2024-02-13 20:12:30 [INFO]	[TRAIN] epoch: 4, iter: 220/1000, loss: 0.0964, lr: 0.008005, batch_cost: 0.0710, reader_cost: 0.00018, ips: 56.3217 samples/sec | ETA 00:00:55
2024-02-13 20:12:31 [INFO]	[TRAIN] epoch: 4, iter: 230/1000, loss: 0.1316, lr: 0.007913, batch_cost: 0.0750, reader_cost: 0.00019, ips: 53.3362 samples/sec | ETA 00:00:57
2024-02-13 20:12:31 [INFO]	[TRAIN] epoch: 4, iter: 240/1000, loss: 0.1088, lr: 0.007821,

76/76 [==============================] - 2s 20ms/step - batch_cost: 0.0199 - reader cost: 3.8186e-04


...

2024-02-13 20:12:59 [INFO]	[EVAL] #Images: 76 mIoU: 0.9217 Acc: 0.9970 Kappa: 0.9153 Dice: 0.9577
2024-02-13 20:12:59 [INFO]	[EVAL] Class IoU: 
[0.9969 0.8465]
2024-02-13 20:12:59 [INFO]	[EVAL] Class Precision: 
[0.9982 0.9325]
2024-02-13 20:12:59 [INFO]	[EVAL] Class Recall: 
[0.9988 0.9017]
2024-02-13 20:12:59 [INFO]	[EVAL] The model with the best validation mIoU (0.9217) was saved at iter 600.
2024-02-13 20:13:00 [INFO]	[TRAIN] epoch: 10, iter: 610/1000, loss: 0.0731, lr: 0.004295, batch_cost: 0.0692, reader_cost: 0.00020, ips: 57.8029 samples/sec | ETA 00:00:26
2024-02-13 20:13:01 [INFO]	[TRAIN] epoch: 10, iter: 620/1000, loss: 0.0704, lr: 0.004196, batch_cost: 0.0683, reader_cost: 0.00019, ips: 58.5688 samples/sec | ETA 00:00:25
2024-02-13 20:13:01 [INFO]	[TRAIN] epoch: 10, iter: 630/1000, loss: 0.0644, lr: 0.004097, batch_cost: 0.0688, reader_cost: 0.00018, ips: 58.1560 samples/sec | ETA 00:00:25
2024-02-13 20:13:02 [INFO]	[TRAIN] epoch: 10, iter: 640/1000, loss: 0.0645, lr: 0.003

76/76 [==============================] - 1s 20ms/step - batch_cost: 0.0192 - reader cost: 3.5140e-04


2024-02-13 20:13:14 [INFO]	[EVAL] #Images: 76 mIoU: 0.9202 Acc: 0.9970 Kappa: 0.9135 Dice: 0.9568
2024-02-13 20:13:14 [INFO]	[EVAL] Class IoU: 
[0.9969 0.8435]
2024-02-13 20:13:14 [INFO]	[EVAL] Class Precision: 
[0.998  0.9409]
2024-02-13 20:13:14 [INFO]	[EVAL] Class Recall: 
[0.999  0.8907]
2024-02-13 20:13:15 [INFO]	[EVAL] The model with the best validation mIoU (0.9217) was saved at iter 600.
2024-02-13 20:13:15 [INFO]	[TRAIN] epoch: 13, iter: 810/1000, loss: 0.0622, lr: 0.002254, batch_cost: 0.0720, reader_cost: 0.00019, ips: 55.5784 samples/sec | ETA 00:00:13
2024-02-13 20:13:16 [INFO]	[TRAIN] epoch: 13, iter: 820/1000, loss: 0.0608, lr: 0.002147, batch_cost: 0.0689, reader_cost: 0.00018, ips: 58.0797 samples/sec | ETA 00:00:12
2024-02-13 20:13:17 [INFO]	[TRAIN] epoch: 13, iter: 830/1000, loss: 0.0605, lr: 0.002040, batch_cost: 0.0689, reader_cost: 0.00017, ips: 58.0778 samples/sec | ETA 00:00:11
2024-02-13 20:13:17 [INFO]	[TRAIN] epoch: 13, iter: 840/1000, loss: 0.0615, lr: 0.001

76/76 [==============================] - 1s 19ms/step - batch_cost: 0.0191 - reader cost: 3.6953e-04


2024-02-13 20:13:30 [INFO]	[EVAL] #Images: 76 mIoU: 0.9200 Acc: 0.9970 Kappa: 0.9133 Dice: 0.9567
2024-02-13 20:13:30 [INFO]	[EVAL] Class IoU: 
[0.9969 0.8431]
2024-02-13 20:13:30 [INFO]	[EVAL] Class Precision: 
[0.9978 0.948 ]
2024-02-13 20:13:30 [INFO]	[EVAL] Class Recall: 
[0.9991 0.8839]
2024-02-13 20:13:30 [INFO]	[EVAL] The model with the best validation mIoU (0.9217) was saved at iter 600.
<class 'paddle.nn.layer.conv.Conv2D'>'s flops has been counted
<class 'paddle.nn.layer.norm.BatchNorm2D'>'s flops has been counted
<class 'paddle.nn.layer.activation.ReLU'>'s flops has been counted
<class 'paddle.nn.layer.pooling.AvgPool2D'>'s flops has been counted
<class 'paddle.nn.layer.pooling.AdaptiveAvgPool2D'>'s flops has been counted
Total Flops: 6339220352     Total Params: 8214802


### 恢复训练

如果需要恢复训练，只需指定参数resume_model即可。

In [ ]:
from paddleseg.core import train
from paddleseg.models import PPLiteSeg
from paddleseg.models.backbones import STDC1

model = PPLiteSeg(
    num_classes=2, # 背景+前景类别数
    backbone=STDC1()
)
train(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    optimizer=optimizer,
    save_dir='output',
    iters=1000,
    batch_size=4,
    save_interval=200,
    log_iters=10,
    num_workers=0,
    losses=losses,
    resume_model='output/iter_200',
    use_vdl=True)

## 2.2 模型评估与预测

PaddleSeg在训练时是以[paddle基础API保存模型](https://www.paddlepaddle.org.cn/documentation/docs/zh/guides/beginner/model_save_load_cn.html)，因此在加载模型参数时也对应使用paddle基础API。

### 模型评估

paddleseg.core提供了模型评估的API，指定参数后即可进行评估。

In [10]:
# 开始评估
import paddle
model_path = 'output/best_model/model.pdparams'
if model_path:
    para_state_dict = paddle.load(model_path)
    model.set_dict(para_state_dict)
    print('Loaded trained params of model successfully')
else: 
    raise ValueError('The model_path is wrong: {}'.format(model_path))

from paddleseg.core import evaluate
evaluate(
    model,
    val_dataset
)

Loaded trained params of model successfully
2024-02-13 20:14:00 [INFO]	Start evaluating (total_samples: 76, total_iters: 76)...


/root/miniconda3/envs/paddle_learn/lib/python3.8/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for seg_heads.1.conv._conv.total_ops. seg_heads.1.conv._conv.total_ops is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
/root/miniconda3/envs/paddle_learn/lib/python3.8/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for seg_heads.1.conv._conv.total_params. seg_heads.1.conv._conv.total_params is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
/root/miniconda3/envs/paddle_learn/lib/python3.8/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for seg_heads.1.conv._batch_norm.total_ops. seg_heads.1.conv._batch_norm.total_ops is not found in the provided dict.
  warnings.warn(f"Skip loading for {key}. " + str(err))
/root/miniconda3/envs/paddle_learn/lib/python3.8/site-packages/paddle/nn/layer/layers.py:2084: UserWarning: Skip loading for seg_head

2024-02-13 20:14:01 [INFO]	[EVAL] #Images: 76 mIoU: 0.9217 Acc: 0.9970 Kappa: 0.9153 Dice: 0.9577
2024-02-13 20:14:01 [INFO]	[EVAL] Class IoU: 
[0.9969 0.8465]
2024-02-13 20:14:01 [INFO]	[EVAL] Class Precision: 
[0.9982 0.9325]
2024-02-13 20:14:01 [INFO]	[EVAL] Class Recall: 
[0.9988 0.9017]


(0.9217038174435206,
 0.9969968844186231,
 array([0.99694633, 0.84646131]),
 array([0.99816234, 0.93251858]),
 0.9153183253929513)

In [11]:
# 模型多尺度+翻转评估
evaluate(
        model,
        val_dataset,
        aug_eval=True,
        scales=[0.75, 1.0, 1.25],
        flip_horizontal=True)

2024-02-13 20:14:19 [INFO]	Start evaluating (total_samples: 76, total_iters: 76)...


76/76 [==============================] - 7s 86ms/step - batch_cost: 0.0850 - reader cost: 4.2746e-04


2024-02-13 20:14:26 [INFO]	[EVAL] #Images: 76 mIoU: 0.9258 Acc: 0.9972 Kappa: 0.9201 Dice: 0.9600
2024-02-13 20:14:26 [INFO]	[EVAL] Class IoU: 
[0.9971 0.8544]
2024-02-13 20:14:26 [INFO]	[EVAL] Class Precision: 
[0.9981 0.9438]
2024-02-13 20:14:26 [INFO]	[EVAL] Class Recall: 
[0.999  0.9003]


(0.9257831669139771,
 0.9971838110392872,
 array([0.99713648, 0.85442986]),
 array([0.99813583, 0.94377945]),
 0.9200684014326543)

### 模型预测

paddleseg.core提供了模型预测的API，在预测时需传入待预测的图像列表。

In [12]:
# 构建待预测的图像列表
import os
def get_image_list(image_path):
    """Get image list"""
    valid_suffix = [
        '.JPEG', '.jpeg', '.JPG', '.jpg', '.BMP', '.bmp', '.PNG', '.png'
    ]
    image_list = []
    image_dir = None
    if os.path.isfile(image_path):
        if os.path.splitext(image_path)[-1] in valid_suffix:
            image_list.append(image_path)
    elif os.path.isdir(image_path):
        image_dir = image_path
        for root, dirs, files in os.walk(image_path):
            for f in files:
                if os.path.splitext(f)[-1] in valid_suffix:
                    image_list.append(os.path.join(root, f))
    else:
        raise FileNotFoundError(
            '`--image_path` is not found. it should be an image file or a directory including images'
        )

    if len(image_list) == 0:
        raise RuntimeError('There are not image file in `--image_path`')

    return image_list, image_dir
image_path = 'data/optic_disc_seg/JPEGImages/N0010.jpg' # 也可以输入一个包含图像的目录
image_list, image_dir = get_image_list('data/optic_disc_seg/JPEGImages/N0010.jpg')

指定predict函数所需参数后即可进行预测，预测后的结果图保存在save_dir指定的目录中。

In [13]:
# 开始预测
from paddleseg.core import predict
import paddleseg.transforms as T
from paddleseg.models import PPLiteSeg
from paddleseg.models.backbones import STDC1

model = PPLiteSeg(
    num_classes=2, # 背景+前景类别数
    backbone=STDC1()
)
pred_transforms = T.Compose([
    T.Resize(target_size=(512, 512)),
    T.Normalize()
])

predict(
        model,
        model_path='output/best_model/model.pdparams',
        transforms=pred_transforms,
        image_list=image_list,
        image_dir=image_dir,
        save_dir='output/results'
    )

2024-02-13 20:16:13 [INFO]	Loading pretrained model from output/best_model/model.pdparams
2024-02-13 20:16:13 [INFO]	There are 250/250 variables loaded into PPLiteSeg.
2024-02-13 20:16:13 [INFO]	Start to predict...


1/1 [==============================] - 0s 37ms/step


2024-02-13 20:16:13 [INFO]	Predicted images are saved in output/results/added_prediction and output/results/pseudo_color_prediction .


### 预测结果可视化

预测的伪彩色图如下：

![](https://ai-studio-static-online.cdn.bcebos.com/f373b993061843988434d80c82715597a491477c655a40f7a426d18dac9472aa)


叠加结果图如下：

![](https://ai-studio-static-online.cdn.bcebos.com/a4db253645d34ce09f8ae45e08c59bfae69d90e1f4c34f978333031484facaa5)